# Load Packages and Data

In [2]:
using CSV, DataFrames, JuMP, Gurobi;

# Model Solver

In [13]:
function solve_fantasy_football_model(; 
        initial_squad::Vector{Int64},
        weeks::UnitRange{Int64}, 
        triple_captains_remaining::Float64, 
        bench_boost_remaining::Float64, 
        wildcard_remaining::Float64, 
        freehit_remaining::Float64
    )

    # initialize model
    model = Model(Gurobi.Optimizer);
    
    # players in squad
    @variable(model, S[players, weeks], Bin)
    
    # players in team
    @variable(model, X[players, weeks], Bin)
    
    # captain and triple captain
    @variable(model, C[players, weeks], Bin)
    @variable(model, TC[players, weeks], Bin)
    
    # bench boost
    @variable(model, BB[weeks], Bin)
    
    # transfer tracker
    @variable(model, Tr[players, weeks] >= 0, Bin)
    @variable(model, TTr[weeks] >= 0, Int)
    transfer_weeks = (minimum(weeks) + 1) : maximum(weeks)
    
    # carry forward transfers
    @variable(model, CarryForwardTransfers[weeks] >= 0, Int)
    
    # wildcard
    @variable(model, WC[weeks], Bin)
    
    # free hit
    @variable(model, FH[weeks], Bin)
    
    # objective function
    @objective(
        model, 
        Max, 
        sum(sum(X[i, j] * XP[i, j] for i in players) for j in weeks)
        +
        sum(sum(C[i, j] * XP[i, j] for i in players) for j in weeks)
        +
        sum(sum(TC[i, j] * XP[i, j] for i in players) for j in weeks)
        +
        sum(sum((S[i, j] - X[i, j]) * BB[j] * XP[i, j] for i in players) for j in weeks) 
        -
        4 * sum(TTr[j] * (1 - WC[j] - FH[j]) for j in transfer_weeks)
    )

    # starting squad from prior iteration
    if sum(initial_squad) > 0
        for i in initial_squad
            @constraint(model, S[i, 1] == 1)
        end
    end
 
    # salary cap
    @constraint(model, [j in weeks], sum(P[i] * S[i, j] for i in players) <= 100)
    
    # number of players in squad
    @constraint(model, [j in weeks], sum(GK[i] * S[i, j] for i in players) == 2)
    @constraint(model, [j in weeks], sum(DEF[i] * S[i, j] for i in players) == 5)
    @constraint(model, [j in weeks], sum(MID[i] * S[i, j] for i in players) == 5)
    @constraint(model, [j in weeks], sum(FWD[i] * S[i, j] for i in players) == 3)
    
    # number of players in team
    @constraint(model, [j in weeks], sum(X[i, j] for i in players) == 11)
    @constraint(model, [j in weeks], sum(GK[i] * X[i, j] for i in players) == 1)
    @constraint(model, [j in weeks], sum(DEF[i] * X[i, j] for i in players) >= 3)
    @constraint(model, [j in weeks], sum(MID[i] * X[i, j] for i in players) >= 2)
    @constraint(model, [j in weeks], sum(FWD[i] * X[i, j] for i in players) >= 1)
    
    # players in team must be in squad
    @constraint(model, [j in weeks, i in players], X[i, j] <= S[i, j])
    
    # number of players from each team
    @constraint(model, [j in weeks], sum(ARS[i] * S[i, j] for i in players) <= 3)
    @constraint(model, [j in weeks], sum(CHE[i] * S[i, j] for i in players) <= 3)
    @constraint(model, [j in weeks], sum(LIV[i] * S[i, j] for i in players) <= 3)
    @constraint(model, [j in weeks], sum(MCI[i] * S[i, j] for i in players) <= 3)
    @constraint(model, [j in weeks], sum(MUN[i] * S[i, j] for i in players) <= 3)
    @constraint(model, [j in weeks], sum(TOT[i] * S[i, j] for i in players) <= 3)
    
    # one captain per week
    @constraint(model, [j in weeks], sum(C[i, j] for i in players) == 1)
    @constraint(model, [j in weeks, i in players], C[i, j] <= X[i, j])
    
    # one triple captain in season
    @constraint(model, sum(sum(TC[i, j] for i in players) for j in weeks) == triple_captains_remaining)
    @constraint(model, [j in weeks, i in players], TC[i, j] <= C[i, j])
    
    # one bench boost in season
    @constraint(model, sum(BB[j] for j in weeks) == bench_boost_remaining)
    
    # track transfers
    @constraint(model, [j in transfer_weeks, i in players], Tr[i, j] >= S[i, j] - S[i, j-1])
    @constraint(model, [j in transfer_weeks], TTr[j] == sum(Tr[i, j] for i in players) - 1 - CarryForwardTransfers[j])
    
    # carry forward transfers
    @constraint(model, CarryForwardTransfers[1] == 0)
    @constraint(model, [j in 2:maximum(weeks)], CarryForwardTransfers[j] <= 1)
    @constraint(model, [j in 2:maximum(weeks)], CarryForwardTransfers[j] >= 0)
    @constraint(model, [j in 2:maximum(weeks)], CarryForwardTransfers[j] <= CarryForwardTransfers[j-1] + 1 - TTr[j])
    
    # wildcards
    @constraint(model, sum(WC[j] for j in weeks) == wildcard_remaining)
    @constraint(model, [j in 2:maximum(weeks)], WC[j] + WC[j-1] <= 1)
    
    # free hit
    @constraint(model, sum(FH[j] for j in weeks) == freehit_remaining)
    @constraint(model, FH[1] == 0)
    @constraint(model, [j in 2:(maximum(weeks)-1), i in players], S[i, j+1] - S[i, j-1] <= 1 - FH[j])
    @constraint(model, [j in weeks], FH[j] + WC[j] <= 1)
    
    # solve model
    optimize!(model)
    objective_value(model)

    # create list of decision variables
    decision_variables = Dict(
        "S" => value.(S),
        "X" => value.(X),
        "C" => value.(C),
        "TC" => value.(TC),
        "BB" => value.(BB),
        "Tr" => value.(Tr),
        "TTr" => value.(TTr),
        "CarryForwardTransfers" => value.(CarryForwardTransfers),
        "WC" => value.(WC),
        "FH" => value.(FH)
    )

    return decision_variables
end;

# Model Parameters

In [4]:
# load data sets
team = CSV.read("../../data/processed/team.csv", DataFrame)
pos = CSV.read("../../data/processed/position.csv", DataFrame)
price = CSV.read("../../data/processed/price.csv", DataFrame)

# define players
players = 1:nrow(team)

# player positions
GK = pos.PosGK
DEF = pos.PosDEF
MID = pos.PosMID
FWD = pos.PosFWD

# define teams
ARS = team.TeamARS
CHE = team.TeamCHE
LIV = team.TeamLIV
MCI = team.TeamMCI
MUN = team.TeamMUN
TOT = team.TeamTOT

# define prices
P = price.Price;

# Initial Solution

### Fit Model

In [15]:
# extract expected points
points = CSV.read("../../data/processed/points.csv", DataFrame);
XP = points[:, 2:end];

# fit model
decision_variables = solve_fantasy_football_model(
    initial_squad = [0],
    weeks = 1:19, 
    triple_captains_remaining = 1.0, 
    bench_boost_remaining = 1.0, 
    wildcard_remaining = 2.0, 
    freehit_remaining = 1.0
);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-09
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9822 rows, 9785 columns and 36916 nonzeros
Model fingerprint: 0x2d06e3c8
Model has 3862 quadratic objective terms
Variable types: 0 continuous, 9785 integer (9747 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-02, 7e+00]
  QObjective range [3e-02, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 631.4013000
Presolve removed 39 rows and 105 columns
Presolve time: 0.05s
Presolved: 13645 rows, 13542 columns, 48023 nonzeros
Variable types: 0 continuous, 13542 integer (13491 binary)

Root relaxation: objective 1.467323e+03, 21465 iterations, 2.84 se

### Extract Fitted Team

In [16]:
function is_nonzero_row(row)
    return sum(row[2:end]) > 0.0000001
end

function identify_team(decision_variables, n)
    optimal_team = hcat(team.Name[players], decision_variables["X"][:, n])
    optimal_team = filter(row -> is_nonzero_row(row), DataFrame(optimal_team, :auto))
    rename!(optimal_team, :x2 => :Team)
    rename!(optimal_team, :x1 => :Player)
end

function identify_squad(decision_variables, n)
    optimal_squad = hcat(team.Name[players], decision_variables["S"][:, n])
    optimal_squad = filter(row -> is_nonzero_row(row), DataFrame(optimal_squad, :auto))
    rename!(optimal_squad, :x2 => :Squad)
    rename!(optimal_squad, :x1 => :Player)
end;

function identify_captain(decision_variables, n)
    optimal_captain = hcat(team.Name[players], decision_variables["C"][:, n])
    optimal_captain = filter(row -> is_nonzero_row(row), DataFrame(optimal_captain, :auto))
    rename!(optimal_captain, :x2 => :Captain)
    rename!(optimal_captain, :x1 => :Player)
end

function identify_triple_captain(decision_variables, n)
    optimal_triple_captain = hcat(team.Name[players], decision_variables["TC"][:, n])
    optimal_triple_captain = filter(row -> is_nonzero_row(row), DataFrame(optimal_triple_captain, :auto))
    rename!(optimal_triple_captain, :x2 => :TripCaptain)
    rename!(optimal_triple_captain, :x1 => :Player)
end

function compile_results(decision_variables, n)
    
    df1 = identify_squad(decision_variables, n)
    df2 = identify_team(decision_variables, n)
    df3 = identify_captain(decision_variables, n)
    df4 = identify_triple_captain(decision_variables, n)
    
    result = outerjoin(df1, df2, on = :Player) |> x -> outerjoin(x, df3, on = :Player) |> x -> outerjoin(x, df4, on = :Player)
    result = coalesce.(result, 0)
    
    result.BB = fill(sum(decision_variables["BB"][n]), nrow(result))
    result.WC = fill(sum(decision_variables["WC"][n]), nrow(result))
    result.FH = fill(sum(decision_variables["FH"][n]), nrow(result))
    result.TTr = fill(sum(decision_variables["TTr"][n]), nrow(result))
    
    return(result)
end;

In [17]:
# compile results
for i in 1:38
    filename = "../../data/final/single_solution/week_$i.csv"
    df_to_write = compile_results(decision_variables, i)
    CSV.write(filename, df_to_write)
end;

LoadError: KeyError: key 20 not found

# Fit Iterative Models

In [18]:
# initialize chips
triple_captains_remaining = 1.0
bench_boost_remaining = 1.0
wildcard_remaining = 2.0 
freehit_remaining = 1.0

1.0

### Extract Starting Point

In [19]:
# identify squad
S = decision_variables["S"]
num_rows = size(S, 1)
squad = [i for i in 1:num_rows if value(S[i, 1]) == 1]
squad = convert(Array{Int}, squad)

# save starting squad
filename = "../../data/final/iterative_solution/week_1.csv"
df_to_write = compile_results(decision_variables, 1)
CSV.write(filename, df_to_write);

In [20]:
# identify chip usage
triple_captains_remaining -= sum(decision_variables["TC"][:, 1])
bench_boost_remaining -= sum(decision_variables["BB"][1])
wildcard_remaining -= sum(decision_variables["WC"][1])
freehit_remaining -= sum(decision_variables["FH"][1]);

In [21]:
for i in 2:38

    # extract expected points
    points = CSV.read("../../data/processed/predicted_points/without_injuries/points_$(i).csv", DataFrame);
    XP = points[:, 2:end];
    
    # fit model
    decision_variables_it = solve_fantasy_football_model(
        initial_squad = squad,
        weeks = 1:(11 - i), 
        triple_captains_remaining = triple_captains_remaining, 
        bench_boost_remaining = bench_boost_remaining, 
        wildcard_remaining = wildcard_remaining, 
        freehit_remaining = freehit_remaining
    );

    # compile weekly result
    filename = "../../data/final/iterative_solution/week_$i.csv"
    df_to_write = compile_results(decision_variables_it, 2)
    CSV.write(filename, df_to_write)

    filename = "../../data/final/iterative_solution/prev_week_$i.csv"
    df_to_write = compile_results(decision_variables_it, 1)
    CSV.write(filename, df_to_write)
    
    # identify squad
    S = decision_variables_it["S"]
    num_rows = size(S, 1)
    squad = [i for i in 1:num_rows if value(S[i, 2]) == 1]
    squad = convert(Array{Int}, squad)

    # identify chip usage
    triple_captains_remaining -= sum(decision_variables_it["TC"][:, 2])
    bench_boost_remaining -= sum(decision_variables_it["BB"][2])
    wildcard_remaining -= sum(decision_variables_it["WC"][2])
    freehit_remaining -= sum(decision_variables_it["FH"][2])
    
end;

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-09
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4507 rows, 4635 columns and 16961 nonzeros
Model fingerprint: 0x75b45686
Model has 1840 quadratic objective terms
Variable types: 0 continuous, 4635 integer (4617 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-02, 7e+00]
  QObjective range [3e-02, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 453.2765800
Presolve removed 353 rows and 483 columns
Presolve time: 0.05s
Presolved: 5804 rows, 5802 columns, 20254 nonzeros
Variable types: 0 continuous, 5802 integer (5780 binary)

Root relaxation: objective 8.163322e+02, 7904 iterations, 0.77 second

LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.